In [1]:
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 14s 232us/step - loss: 0.0208 - acc: 0.9931
Epoch 2/5
60000/60000 [==============================] - 14s 230us/step - loss: 0.0169 - acc: 0.9947
Epoch 3/5
60000/60000 [==============================] - 14s 232us/step - loss: 0.0154 - acc: 0.9947
Epoch 4/5
60000/60000 [==============================] - 14s 232us/step - loss: 0.0118 - acc: 0.9964
Epoch 5/5
60000/60000 [==============================] - 14s 233us/step - loss: 0.0122 - acc: 0.9960


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  401920    
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.evaluate(x_test, y_test)